In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from delta import *
import pyspark
import os

In [2]:
os.environ["JAVA_HOME"] = "/Users/wonseokchoi/Documents/Installs/jdk-11.0.19.jdk/Contents/Home"
os.environ["PATH"] = f'{os.environ["JAVA_HOME"]}/bin:{os.environ["PATH"]}'

os.environ["SPARK_HOME"] = "/Users/wonseokchoi/Documents/Installs/spark-3.4.1-bin-hadoop3"
os.environ["PATH"] = f'{os.environ["PATH"]}:{os.environ["SPARK_HOME"]}/bin'
# PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
spark_home = os.environ.get('SPARK_HOME')
if spark_home:
    zips = ":".join(os.path.join(spark_home, "python/lib", f) for f in os.listdir(os.path.join(spark_home, "python/lib")) if f.endswith('.zip'))
    python_path = os.environ.get("PYTHONPATH", "")
    new_python_path = f"{zips}:{python_path}"
    os.environ["PYTHONPATH"] = new_python_path
else:
    print("SPARK_HOME is not set in the environment.")

os.environ["HADOOP_HOME"] = "/Users/wonseokchoi/Documents/Installs/hadoop-3.3.6"
os.environ["PATH"] = f'{os.environ["PATH"]}:{os.environ["HADOOP_HOME"]}/bin'
os.environ["HADOOP_CONF_DIR"] = f'{os.environ["HADOOP_HOME"]}/etc/hadoop'
os.environ["YARN_CONF_DIR"] = f'{os.environ["HADOOP_HOME"]}/etc/hadoop'
os.environ["HADOOP_CLASSPATH"] = f'{os.environ["HADOOP_HOME"]}/share/hadoop/tools/lib/*'

os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

In [3]:
builder = pyspark.sql.SparkSession.builder.appName("AllStocks") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/09/07 22:13:20 WARN Utils: Your hostname, Wonseoks-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.35.2 instead (on interface en0)
23/09/07 22:13:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/wonseokchoi/.ivy2/cache
The jars for the packages stored in: /Users/wonseokchoi/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-06a50fad-e73d-4495-b3a3-b98e19fed463;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central


:: loading settings :: url = jar:file:/Users/wonseokchoi/Documents/Installs/spark-3.4.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 84ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-06a50fad-e73d-4495-b3a3-b98e19fed463
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/3ms)
23/09/07 22:13:21 WARN NativeCodeLoader: Unable to load native-hadoop

In [4]:
df = spark.read \
    .format("delta") \
    .load("../data/lake_replica/all_stocks/")
df1 = df.withColumn("timestamp", F.date_format(F.from_unixtime(F.col("timestamp")/1000), "yyyy-MM-dd HH:mm:ss"))
df_cleaned = df1.drop("updated_at", "time_bucket")
df_googl = df_cleaned.filter(F.col("symbol") == "GOOGL").orderBy(F.col("timestamp").asc())
df_googl.printSchema()
df_googl.show(10)

root
 |-- symbol: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- transactions: long (nullable = true)
 |-- volume: long (nullable = true)
 |-- volume_weighted: long (nullable = true)



23/09/07 22:13:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+-------------------+-------+-------+-------+-------+------------+------+---------------+
|symbol|          timestamp|  close|   high|    low|   open|transactions|volume|volume_weighted|
+------+-------------------+-------+-------+-------+-------+------------+------+---------------+
| GOOGL|2018-09-06 20:00:00|   60.0|   60.0|   60.0|   60.0|           7|  4620|             60|
| GOOGL|2018-09-06 20:53:00|  60.01|  60.01|  60.01|  60.01|           1|  6000|             60|
| GOOGL|2018-09-06 20:54:00| 60.015| 60.015|  60.01|  60.01|           8|  9940|             60|
| GOOGL|2018-09-06 21:00:00|59.9995|59.9995|59.9995|59.9995|          11|  4980|             60|
| GOOGL|2018-09-06 21:03:00| 59.969| 59.969| 59.969| 59.969|           4|  6760|             59|
| GOOGL|2018-09-06 21:15:00|60.0575|60.0575|60.0575|60.0575|           1|  2000|             60|
| GOOGL|2018-09-06 21:22:00|60.0575|60.0575| 60.052|60.0575|          16| 14140|             60|
| GOOGL|2018-09-06 21:31:00|60

In [5]:
DF = df_googl.toPandas()
DF.head()

,symbol,timestamp,close,high,low,open,transactions,volume,volume_weighted
0,GOOGL,2018-09-06 20:00:00,60.0000,60.0000,60.0000,60.0000,7.0,4620.0,60.0
1,GOOGL,2018-09-06 20:53:00,60.0100,60.0100,60.0100,60.0100,1.0,6000.0,60.0
2,GOOGL,2018-09-06 20:54:00,60.0150,60.0150,60.0100,60.0100,8.0,9940.0,60.0
3,GOOGL,2018-09-06 21:00:00,59.9995,59.9995,59.9995,59.9995,11.0,4980.0,60.0
4,GOOGL,2018-09-06 21:03:00,59.9690,59.9690,59.9690,59.9690,4.0,6760.0,59.0


In [7]:
DF.to_csv("../data/csv/googl.csv", index=False)